# Implementing the "Cube" approach

In [1]:
import pykep as pk
import numpy as np
import json
import pickle as pkl

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
%matplotlib notebook

import cascade as csc
from copy import deepcopy
from tqdm.notebook import tqdm
import heyoka as hy

# added for the Cube approach implementation
from collections import defaultdict
import time
import sgp4
from sgp4.api import Satrec, SatrecArray

In [2]:
with open("data/debris_simulation_ic.pk", "rb") as file:
    r_ic,v_ic,c_radius,to_satcat,satcat,debris = pkl.load(file)

In [3]:
def period(r,v, mu):
    """Computes the orbital period from the vis-viva equation

    Args:
        r (float): The radius (in L).
        v (float): The velocity (in L/T).
        mu (float): The gravitational parameter in L^3/T^2

    Returns:
        The orbital period (in T)
    """
    En = v**2/2 - mu / r
    a = -mu / En / 2
    if a<0:
        raise ValueError("Hyperbola!!!")
    return np.sqrt(a**3/mu)*2*np.pi

In [4]:
def cubes(cartesian_points, cube_dimension):
    """Runs the Cube algorithm and returns satellites within the same cube

    Args:
        cartesian_points (Nx3 np.array): The cartesian position of the satellites (in L).
        cube_dimension (float): The cube dimentsion (in L).

    Returns:
        a list containing lists of satelites idx occupying the same cube
    """
    # init
    retval = []
    cubes = defaultdict(list)

    # We compute the floored Cartesian coordinates identifying the bins.
    pos = cartesian_points
    pos = pos / cube_dimension
    pos = np.floor(pos).astype(int)
    # We fill the bins
    for i, xyz in enumerate(pos):
        cubes[tuple(xyz)].append(i)
    # We find bins with more than one atellite
    for key in cubes:
        if len(cubes[key]) > 1:
            retval.append(cubes[key])
    return retval

In [5]:
def simulate_sgp4(debris, sim_time=20,time_grid=5, t0 = 8073.603992389981):
    """Computes all satellites ephemerides on a time grid

    Args:
        debris (list of pk.planets): The objects to propagate.
        sim_time (float): The total propagation time (in years).
        time_grid(float): The time resolution (in days).
        t0 (float): the starting epoch in mjd2000.

    Returns:
        a list containing lists of idx identifying the object occupying the same cube
    """
    # This list will contain all the sgp4 Satrec objects
    satellite_l = []
    for deb in debris:
        l1 = deb.line1
        l2 = deb.line2
        satellite_l.append(Satrec.twoline2rv(l1, l2))
    # Here we build the vectorized version allowing for speed
    satellites = SatrecArray(satellite_l)
    jd0, fr = pk.epoch(t0).jd, 0.0
    # The Julian dates are from jd0 to 20 years after
    jds = jd0 + np.arange(0,sim_time*365.25/time_grid)*time_grid
    frs = jds * 0
    return satellites.sgp4(jds, frs)

In [6]:
%timeit cubes(r_ic / 1000, 10)

68 ms ± 604 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
start = time.time()
e,r,v = simulate_sgp4(np.array(debris)[to_satcat], sim_time=20,time_grid=5, t0 = 8073.603992389981+np.random.random()*5)
end = time.time()
print("Time elasped: ", end - start)

Time elasped:  21.077648401260376


In [8]:
# We assume all satellites are valid at the starting epoch
start = time.time()
undecayed = set(np.arange(r.shape[0]))
for i in range(r.shape[1]):
    Lcube = 10. #km
    time_grid  = 5 #days
    # If signalled from the sgp4, we remove the indices of the decayed satellites
    decayed = set(np.where(e[:,i]>0)[0])
    undecayed = undecayed - decayed
    undecayed_l = np.array([j for j in undecayed])
    # We detect all satellites couples in the same cube of Lcube km size
    collision = cubes(r[undecayed_l,i,:], cube_dimension = Lcube)
    for pair in collision:
        # we get the indexes in r,v
        idx1 = undecayed_l[pair[0]]
        idx2 = undecayed_l[pair[1]]
        # we store positions and velocities from r,v
        r1 = r[idx1,i,:]
        r2 = r[idx2,i,:]
        v1 = v[idx1,i,:]
        v2 = v[idx2,i,:]
        # we get the collision radiu from debris (indexed differently hence to_satcat is used)
        c_radius1 = debris[to_satcat[idx1]].collision_radius
        c_radius2 = debris[to_satcat[idx2]].collision_radius
        # Relative velocity 
        Vrel = np.linalg.norm(v1-v2)
        # Collisional area of the couple (in km^2)
        sigma = np.pi*((c_radius1+c_radius2)/1000)**2 
        # Volume of the cube (km^3)
        U = Lcube**3
        # We compute the spatial densities
        # Time spent in the cube (nd)
        tau1 = Lcube / np.linalg.norm(v1)
        tau2 = Lcube / np.linalg.norm(v2)
        # Orbital periods (s)
        T1 = period(np.linalg.norm(r1*1000),np.linalg.norm(v1*1000), pk.MU_EARTH)
        T2 = period(np.linalg.norm(r2*1000), np.linalg.norm(v2*1000), pk.MU_EARTH)
        # densities (from "Assessing collision algorithms for the newspace era" )
        s1 = 1./U
        s2 = 1./U
        # collision probability
        Pij = s1*s2*Vrel*sigma*U*time_grid*pk.DAY2SEC
        # Store
        if Pij > np.random.random():
            print(f"Collision! pair: {pair}, years: {i*5/365.25}")
end = time.time()
print("Time elasped: ", end - start)

Collision! pair: [3328, 7009], years: 0.4517453798767967
Collision! pair: [13649, 15915], years: 0.6844626967830253
Collision! pair: [14085, 14154], years: 3.3675564681724848
Collision! pair: [13319, 14347], years: 4.339493497604381
Collision! pair: [12008, 16440], years: 6.652977412731006
Collision! pair: [13222, 13925], years: 7.693360711841205
Collision! pair: [13360, 14870], years: 9.678302532511978
Collision! pair: [14120, 14713], years: 11.07460643394935
Collision! pair: [7062, 15201], years: 11.594798083504449
Collision! pair: [10347, 15379], years: 12.14236824093087
Collision! pair: [13811, 13991], years: 12.4435318275154
Collision! pair: [3938, 14782], years: 14.907597535934292
Collision! pair: [1618, 14789], years: 15.66050650239562
Time elasped:  118.71472120285034
